In [13]:
import math

class Vec2:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
        
    def diff(self, other: 'Vec2') -> 'Vec2':
        return Vec2(self.x - other.x, self.y - other.y)
    
    def magnitude(self) -> float:
        return math.sqrt(self.x * self.x + self.y * self.y)
    
    def unit(self) -> 'Vec2':
        m = self.magnitude()
        if m <= 0:
            return Vec2(0.0, 0.0)
        return Vec2(self.x / m, self.y / m)
    
    def dot(self, other: 'Vec2') -> float:
        return self.x * other.x + self.y * other.y
    
    def angle(self, other: 'Vec2') -> float:
        """Get the angle between self and other."""
        d = self.dot(other)
        return math.acos(d)
    
    def rotated(self, rads: float) -> 'Vec2':
        """Get self, rotated clockwise by rads."""
        cos = math.cos(rads)
        sin = math.sin(rads)
        x = self.x * cos - self.y * sin
        y = self.y * c + self.x * sin
        return Vec2(x, y)
    
    def normal(self) -> 'Vec2':
        """Get a unit Vec2 that is normal to self."""
        # let self be a unit vector (x1, y1).
        # let the result be a unit vector (x2, y2).
        # Arbitrarily choose (x2, y2) to be the result of
        # rotating (x1, y1) through 90° counterclockwise.
        u = self.unit()
        angle = math.pi / 2.0
        cos = 0.0  # math.cos(π/2)
        sin = 1.0  # math.sin(π/2)
        x2 = - u.y
        y2 = u.x
        return Vec2(x2, y2)
    
    def scaled(self, magnitude: float) -> 'Vec2':
        return Vec2(self.x * magnitude, self.y * magnitude)
    
    def __str__(self) -> str:
        return f"{self.x},{self.y}"
    
    

In [14]:
v1 = Vec2(1.0, 0.0)
v2 = Vec2(0.0, 1.0)

print(v1.diff(v2).unit())

print(v1.normal())
print(v2.normal())
print("")

def deg(rad: float) -> float:
    return rad * 180.0 / math.pi

print(v1.dot(v2))
print(deg(v1.angle(v2)))
print(deg(v1.angle(v1)))
print(deg(v2.angle(v2)))
print(deg(v2.angle(v1)))

0.7071067811865475,-0.7071067811865475
-0.0,1.0
-1.0,0.0

0.0
90.0
0.0
0.0
90.0


In [50]:
class UnitParticle:
    def __init__(self, s: Vec2, v: Vec2):
        self.mass = 1.0
        self.radius = 1.0
        self.s = s
        self.v = v
        
    def displacement(self, other: 'UnitParticle') -> Vec2:
        return other.s.diff(self.s)
    
    def rel_vel(self, other: 'UnitParticle') -> Vec2:
        return self.v.diff(other.v)
    
    def is_colliding(self, other: 'UnitParticle') -> bool:
        """Find out if self and other are close enough to be colliding."""
        return self.displacement(other).magnitude() <= (self.radius + other.radius)
    
    def collide(self, other: 'UnitParticle') -> None:
        """
        Model, simplistically, an elastic collision between self and other.
        This method mutates self and other.
        """
        # Don't second-guess whether self.is_colliding(other).
        # Assume no rotation - no "English" in the collision.
        # Assume both particles "reflect" off the line normal to the
        # offset vector between the particles.  Angle of incidence == angle
        # of reflection, and all that.
        
        # If I remembered & understood Newton's Principia, I'd be able to make
        # something of the idea that the instantaneous force on a point particle
        # in a 2-body collision lies along the displacement vector between
        # the two particles.
        # Instead I'll rely on notes from a master's thesis:
        # https://docs.google.com/fileview?id=0Bze6mKYvrpOKYjdkODVhMTAtM2Q4Zi00NzgyLWE2YzMtN2MwZmQ4NjA3OWMw&hl=en
        
        n = self.displacement(other).unit()
        jr = self._calc_impulse(other, n)
        v_new = self.v.diff(n.scaled(-jr / self.mass))
        other_v_new = other.v.diff(n.scaled(jr / other.mass))
        self.v = v_new
        other.v = other_v_new
        
    def impulse(self, other: 'UnitParticle') -> float:
        n = self.displacement(other).unit()
        return self._calc_impulse(other, n)

    def _calc_impulse(self, other: 'UnitParticle', n: Vec2) -> float:
        # Let e, the coefficient of restitution, be 1.  I think that corresponds
        # to a perfectly elastic collision.
        e = 1.0
        # Relative collision velocity:
        vr = self.v.diff(other.v)
        
        numer = -(1.0 + e) * vr.dot(n)
        # Ignore rotation:
        denom = 1.0 / self.mass + 1.0 / other.mass
        return numer / denom
        
    def __str__(self) -> str:
        return f"P(m={self.mass}, r={self.radius}, s={self.s}, v={self.v})"
        
    

In [53]:
p0 = UnitParticle(Vec2(0.0, 0.0), Vec2(0.0, 0.0))
print(p0)
p1 = UnitParticle(Vec2(1.0, 0.0), Vec2(-1.0, 1.0))
print(p1)

print("Displacement:", p0.displacement(p1))
print("Unit displacement:", p0.displacement(p1).unit())
print("Relative velocity:", p0.rel_vel(p1))

P(m=1.0, r=1.0, s=0.0,0.0, v=0.0,0.0)
P(m=1.0, r=1.0, s=1.0,0.0, v=-1.0,1.0)
Displacement: 1.0,0.0
Unit displacement: 1.0,0.0
Relative velocity: 1.0,-1.0


In [54]:
p0.is_colliding(p1)

True

In [55]:
p0.impulse(p1)

-1.0

In [56]:
print("Before:", p0, p1)
p0.collide(p1)
print("After:", p0, p1)

Before: P(m=1.0, r=1.0, s=0.0,0.0, v=0.0,0.0) P(m=1.0, r=1.0, s=1.0,0.0, v=-1.0,1.0)
After: P(m=1.0, r=1.0, s=0.0,0.0, v=-1.0,0.0) P(m=1.0, r=1.0, s=1.0,0.0, v=0.0,1.0)
